# Winepi
A partial implementation of Winepi algorithm described by Mannila, Toivonen and Verkamo in *Discovery of Frequent Episodes in Event Sequences*, 1997.

In [1]:
from episode_miner import Event, EventSequence, Episode, find_sequential_episodes, abs_support, rel_support
from pprint import pprint

In [2]:
sequence_of_events = (Event('a', 1), Event('b', 2), Event('a', 3), Event('a', 5), Event('b', 8))
event_sequence = EventSequence(sequence_of_events=sequence_of_events, start=0, end=9)

frequent_episodes = find_sequential_episodes(event_sequences=event_sequence,
                                             window_width=5, 
                                             min_frequency=0.2, 
                                             only_full_windows=False, 
                                             allow_intermediate_events=True, )
frequent_episodes

[9 ('a',),
 10 ('b',),
 5 ('a', 'a'),
 6 ('a', 'b'),
 4 ('b', 'a'),
 3 ('a', 'b', 'a')]

All episodes in output of ```find_sequential_episodes``` are equipped with ```abs_support```, ```rel_support``` and ```allow_intermediate_events``` attributes.

In [3]:
frequent_episodes[3].abs_support, frequent_episodes[3].rel_support, frequent_episodes[3].allow_intermediate_events

(6, 0.46153846153846156, True)

## Support
Find absolute support for episodes. 

In [4]:
abs_support(event_sequences=event_sequence, 
            episodes=(Episode(('a',)), Episode(('a', 'b'))), 
            window_width=5, 
            only_full_windows=False, 
            allow_intermediate_events=True)

[9, 6]

Find relative support for episodes. 

In [5]:
rel_support(event_sequences=event_sequence, 
            episodes=(Episode(('a',)), Episode(('a', 'b'))), 
            window_width=5, 
            only_full_windows=False, 
            allow_intermediate_events=True)

[0.6923076923076923, 0.46153846153846156]

In both cases, the defaults are
```python 
only_full_windows = False
allow_intermediate_events = episodes.allow_intermediate_events
```
It is more efficient to find the supports for list of episodes than for every episode separetely. 

The side effect of ```abs_support``` and ```rel_support``` is that all the episodes are equipped with ```abs_support```, ```rel_support``` and ```allow_intermediate_events``` attributes.

In [6]:
e = Episode(('a', 'b'))
abs_support(event_sequences=event_sequence, 
            episodes=e, 
            window_width=5, 
            only_full_windows=False, 
            allow_intermediate_events=True)
e.abs_support, e.rel_support, e.allow_intermediate_events

(6, 0.46153846153846156, True)


## Only full windows vs all windows
The default is only_full_windows=False.

In [7]:
sequence_of_events = (Event('a', 0), Event('b', 4), Event('c', 7))
event_sequence = EventSequence(sequence_of_events=sequence_of_events, start=0, end=9)

frequent_episodes = find_sequential_episodes(event_sequence, window_width=5,  min_frequency=0.2, only_full_windows=True)
print('Full windows:', frequent_episodes)
frequent_episodes = find_sequential_episodes(event_sequence, window_width=5,  min_frequency=0.2, only_full_windows=False)
print('All windows: ', frequent_episodes)

Full windows: [2 ('c',), 1 ('a',), 5 ('b',), 1 ('a', 'b'), 2 ('b', 'c')]
All windows:  [5 ('c',), 5 ('a',), 5 ('b',)]


In case of full windows the Winepi frequency of episodes near the start or end of the event sequence is reduced, but the number of all windows is smaller and therefore the relative frequency of episodes is increased.

## Intermediate events vs no intermediate events
An intermediate event is one or more events at the same time. In the next example, the only episode which has an a intermediate event is ('a', 'd').

The default is 

```python 
no_intermediate_events=False
```
In the next example, the only episode which has an intermediate event is ```('a', 'd')```.

In [8]:
sequence_of_events = (Event('a', 0), Event('b', 2), Event('c', 2), Event('d', 3))
event_sequence = EventSequence(sequence_of_events=sequence_of_events, start=0, end=6)

print('Allow intermediate events:')
frequent_episodes = find_sequential_episodes(event_sequence, 
                                             window_width=4, min_frequency=0.1, 
                                             allow_intermediate_events=False)
pprint(frequent_episodes)
print('No intermediate events:')
frequent_episodes = find_sequential_episodes(event_sequence, 
                                             window_width=4, min_frequency=0.1, 
                                             allow_intermediate_events=True)
pprint(frequent_episodes)

Allow intermediate events:
[4 ('d',),
 4 ('c',),
 4 ('a',),
 4 ('b',),
 3 ('c', 'd'),
 2 ('a', 'c'),
 2 ('a', 'b'),
 3 ('b', 'd'),
 1 ('a', 'c', 'd'),
 1 ('a', 'b', 'd')]
No intermediate events:
[4 ('d',),
 4 ('c',),
 4 ('a',),
 4 ('b',),
 3 ('c', 'd'),
 1 ('a', 'd'),
 2 ('a', 'c'),
 2 ('a', 'b'),
 3 ('b', 'd'),
 1 ('a', 'c', 'd'),
 1 ('a', 'b', 'd')]


In the next example "allow intermediate events" version finds no pattern, but "no intermediate events" version returns even episodes which occur only once (e.g. ('d', 'b')).

In [9]:
sequence_of_events = (Event('a', 1), Event('c',  2), Event('b',  3), Event('c',  4), 
                      Event('a', 5), Event('d',  6), Event('b',  7), Event('d',  8), 
                      Event('a', 9), Event('e', 10), Event('b', 11), Event('e', 12)) 
event_sequence = EventSequence(sequence_of_events=sequence_of_events, start=0, end=13)

frequent_episodes = find_sequential_episodes(event_sequence, 
                                             window_width=4, min_frequency=0.2, 
                                             allow_intermediate_events=True)
print('Alloow intermediate events:')
pprint(frequent_episodes)
frequent_episodes = find_sequential_episodes(event_sequence, 
                                             window_width=4, min_frequency=0.2, 
                                             allow_intermediate_events=False)
print('\nNo intermediate events:')
pprint(frequent_episodes)

Alloow intermediate events:
[6 ('d',),
 6 ('c',),
 12 ('a',),
 6 ('e',),
 12 ('b',),
 4 ('d', 'b'),
 4 ('c', 'b'),
 6 ('a', 'b'),
 4 ('b', 'd'),
 4 ('b', 'a'),
 4 ('b', 'e')]

No intermediate events:
[6 ('d',), 6 ('c',), 12 ('a',), 6 ('e',), 12 ('b',)]
